In [117]:
from IPython.core.display import display, HTML #für eine Breitansicht des Jupyter Notebooks
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
#from xgboost import XGBClassifier

train = pd.read_csv('/Users/patrickbaongu/Desktop/Titanic/train.csv')
test = pd.read_csv('/Users/patrickbaongu/Desktop/Titanic/test.csv')

#print(train.isnull().sum())
#print(test.isnull().sum())
#train.info()
# def bar_chart(feature):
#
#     survived = train[train['Survived'] == 1][feature].value_counts()
#     dead = train[train['Survived'] == 0][feature].value_counts()
#     df = pd.DataFrame([survived, dead])
#     df.index = ['Survived', 'Dead']
#     df.plot(kind = 'bar', stacked = 'True', figsize=(15,8), title = feature)


train_test_data = [train, test]
pat = '([A-Za-z]+)\.' #see doc of 'regular expressions'
for dataset in train_test_data:

    dataset['Title'] = dataset['Name'].str.extract(pat, expand = False)

sex_mapping = {'male':0, 'female':1}

drop_features = ['Name','SibSp', 'Parch', 'Ticket', 'Cabin']

for data in train_test_data:

    #title_mapping = {'Mr':0, 'Mrs':1/2, 'Miss': 1}
    embark_to_num = {'S':0, 'C':1/2, 'Q':1}

    i = 0


    for title in data['Title'].unique()[3:]:

        title_mapping.update({title: 3})

    data['Title'] = data['Title'].map(title_mapping)
    data['Sex'] = data['Sex'].map(sex_mapping)

    data['Age'].fillna(data.groupby('Title')['Age'].transform('mean'), inplace = True)
    data['Age'] = pd.cut(data['Age'], bins = [data['Age'].min(),18,28,38,48,data['Age'].max()],labels = np.linspace(0,1,5), include_lowest = True)
    #bins give the intervals of bin, labels assigns corresponding label

    data['Embarked'] = data['Embarked'].map(embark_to_num)
    data['Embarked'].fillna(data.groupby('Pclass')['Embarked'].transform('median'), inplace = True)


    data['Fare'].replace(0,np.nan)
    data['Fare'].fillna(data.groupby('Pclass')['Fare'].transform('mean'), inplace = True)

    data['Fare'] = pd.cut(data['Fare'], bins = [data['Fare'].min(),10,25,45,60,data['Fare'].max()], labels = np.linspace(0,1,5), include_lowest = True)


    data['Family Size'] = 1 + data['SibSp'] + data['Parch']
    #data['Family Size'] = pd.cut(data['Family Size'], bins = [0,1,4, data['Family Size'].max()], labels = np.linspace(0,1,3), include_lowest = True)


In [118]:
train = pd.get_dummies(train, prefix = ['Family Size', 'Title', 'Embarked', 'Age', 'Fare', 'Pclass'], columns = ['Family Size', 'Title', 'Embarked', 'Age', 'Fare', 'Pclass'])
test = pd.get_dummies(test, prefix = ['Family Size', 'Title', 'Embarked','Age', 'Fare', 'Pclass'], columns = ['Family Size', 'Title', 'Embarked','Age', 'Fare', 'Pclass'])


train.drop('PassengerId', axis = 1, inplace = True)
train.drop(drop_features, axis = 1, inplace = True)
test.drop(drop_features, axis = 1, inplace = True)
train_data = train.drop('Survived', axis = 1)
labels = train['Survived']


In [119]:
train.head()


,Survived,Sex,Family Size_1,Family Size_2,Family Size_3,Family Size_4,Family Size_5,Family Size_6,Family Size_7,Family Size_8,...,Age_0.75,Age_1.0,Fare_0.0,Fare_0.25,Fare_0.5,Fare_0.75,Fare_1.0,Pclass_1,Pclass_2,Pclass_3
0,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
2,1,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [120]:
test.head()

,PassengerId,Sex,Family Size_1,Family Size_2,Family Size_3,Family Size_4,Family Size_5,Family Size_6,Family Size_7,Family Size_8,...,Age_0.75,Age_1.0,Fare_0.0,Fare_0.25,Fare_0.5,Fare_0.75,Fare_1.0,Pclass_1,Pclass_2,Pclass_3
0,892,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,893,1,0,1,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
2,894,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
3,895,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,896,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [121]:

total = []
for k in range(5,15): #Check in how many sets we should split for cross validation

    kfold = KFold(n_splits = k, shuffle = True, random_state = 42)
    models = [KNeighborsClassifier(n_neighbors = 12), DecisionTreeClassifier(), RandomForestClassifier(), GaussianNB(), SVC()]
    scores = []
    for model in models:

        cv_score = cross_val_score(estimator = model, X = train_data, y = labels, cv = kfold, n_jobs =1, scoring = 'accuracy')
        rounded = round(np.mean(cv_score),4)
        scores.append(rounded)
    max_idx = np.argmax(scores)
    total.append((k,np.sum(scores)))
k = max(total, key = lambda x: x[1])[0]
print('We do a {}-fold cross validation.'.format(k))
#Check how many neighbors to chose optimally
kfold = KFold(n_splits = k, shuffle = True, random_state = 42)
knn_scores = []
for i in range(10,20):
    cv_score = cross_val_score(estimator = KNeighborsClassifier(n_neighbors = i), X = train_data, y = labels, cv = kfold, n_jobs =1, scoring = 'accuracy')
    rounded = round(np.mean(cv_score),4)
    knn_scores.append((i,rounded))
optimal_knn = max(knn_scores, key = lambda x: x[1])[0]
print('For kNN we chose {} neighbors.'.format(optimal_knn))

kfold = KFold(n_splits = k, shuffle = True, random_state = 42)
models = [KNeighborsClassifier(n_neighbors = optimal_knn), DecisionTreeClassifier(), RandomForestClassifier(), GaussianNB(), SVC()]
scores = []
for model in models:

    cv_score = cross_val_score(estimator = model, X = train_data, y = labels, cv = kfold, n_jobs =1, scoring = 'accuracy')
    rounded = round(np.mean(cv_score),4)
    scores.append(rounded)

max_idx = np.argmax(scores)
scores

# for col in ['Age', 'Fare', 'Family Size']:
#     test[col] = pd.to_numeric(test[col])
test_data = test.drop('PassengerId', axis =1).copy()
clf = models[max_idx].fit(train_data, labels)
y_pred = clf.predict(test_data)

submission = pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':y_pred})
submission.to_csv('submission.csv', index = False)
submission = pd.read_csv('submission.csv')

print('{} passengers survived.'.format(submission['Survived'].sum()))
print(submission.head(15))



We do a 5-fold cross validation.
For kNN we chose 18 neighbors.
149 passengers survived.
    PassengerId  Survived
0           892         0
1           893         0
2           894         0
3           895         0
4           896         1
5           897         0
6           898         1
7           899         0
8           900         1
9           901         0
10          902         0
11          903         0
12          904         1
13          905         0
14          906         1
